## Construct DataSet

In [ ]:
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader

training_data = datasets.FashionMNIST(
    root="./data",
    train=True,
    download=True,
    transform=transforms.ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="./data",
    train=False,
    download=True,
    transform=transforms.ToTensor(),
)

batch_size = 128
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# Define Model

In [ ]:
import torch
from torch import nn

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # input: 1*28*28
        self.cnn_stack = nn.Sequential(
            nn.Conv2d(1, 6, 3),  # 6*26*26
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 6*13*13
            nn.Dropout(0.2),
            nn.Conv2d(6, 16, 3), # 16*11*11
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 16*5*5
            nn.Dropout(0.2),
            nn.Flatten(),
            nn.Linear(16*5*5, 210),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(210, 90),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(90, 10),
            nn.Dropout(0.2),
            nn.ReLU(),
        )

    def forward(self, x):
        logits = self.cnn_stack(x)
        return logits

model = CNN().to(device)

# Loss function & Optimizer

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.03)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 50
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")